## Preamble

In [ ]:
%load_ext autoreload
%autoreload 0

In [ ]:
%autoreload

In [ ]:
import xarray as xr
import sqlite3
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import torch
import pyro
import scipy as sp

import lib.plot
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.cluster import AgglomerativeClustering
from lib.pandas_util import idxwhere


import sfacts as sf

# from lib.project_style import color_palette, major_allele_frequency_bins
# from lib.project_data import metagenotype_db_to_xarray
# from lib.plot import ordination_plot, mds_ordination, nmds_ordination
# import lib.plot
# from lib.plot import construct_ordered_pallete
# from lib.pandas_util import idxwhere

## Load Data

In [ ]:
fit = sf.data.World.load('data/zshi.sp-102492.metagenotype.filt-poly05-cvrg25.fit-sfacts13-s500-g500-seed3.world.nc')
fit.sizes

In [ ]:
ref = sf.data.Metagenotypes.load('data/gtprodb.sp-102492.genotype.nc').mlift('sel', position=fit.position).to_estimated_genotypes(pseudo=0)
ref.sizes

In [ ]:
bins = np.linspace(0, 1, num=51)

plt.hist(fit.metagenotypes.to_estimated_genotypes(pseudo=1e-3).entropy(), bins=bins)
plt.hist(fit.genotypes.entropy(), bins=bins)
plt.hist(ref.entropy(), bins=bins)

plt.yscale('log')
None

In [ ]:
bins = np.linspace(0, 1, num=51)

fig, axs = plt.subplots(2)
axs[0].hist(fit.genotypes.values.flatten(), bins=bins)
axs[1].hist(fit.communities.max("strain").values.flatten(), bins=bins)

for ax in axs:
    ax.set_yscale('log')
None

In [ ]:
sf.plot.plot_genotype(fit, scaley=2e-2, scalex=1e-3, yticklabels=0)

In [ ]:
sf.plot.plot_genotype(ref, scaley=1e-2, scalex=1e-3, yticklabels=0)

In [ ]:
fit_genotypes_filt = fit.genotypes.mlift('sel', strain=fit.genotypes.entropy() < 0.25)
fit_genotypes_highent = fit.genotypes.mlift('sel', strain=fit.genotypes.entropy() > 0.25)

fit_genotypes_filt.sizes, fit_genotypes_highent.sizes

In [ ]:
plt.hist(squareform(fit_genotypes_filt.pdist()), bins=np.linspace(0, 0.5, num=101))
plt.yscale('log')
None

In [ ]:
g = sf.data.Genotypes.concat(dict(
    bad=fit_genotypes_highent,
    good=fit_genotypes_filt
), dim='strain')


sf.plot.plot_genotype(g, row_colors_func=lambda w: w.strain.str.startswith('good'), scaley=2e-2, scalex=1e-3, yticklabels=0)

In [ ]:
g = sf.data.Genotypes.concat(dict(
    ref=ref,
    fit=fit_genotypes_filt,
    ent=fit_genotypes_highent,
), dim='strain')


sf.plot.plot_genotype(g, row_colors_func=lambda w: w.strain.str.startswith('fit'), scaley=2e-2, scalex=1e-3, yticklabels=0)

In [ ]:
g = sf.data.Genotypes.concat(dict(
    ref=ref,
    fit=fit_genotypes_filt,
), dim='strain')


sf.plot.plot_genotype(g, row_colors_func=lambda w: w.strain.str.startswith('fit'), scaley=2e-2, scalex=1e-3, yticklabels=0)